In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import traceback
import re
import time

firefox_profile=webdriver.FirefoxProfile('/home/suzie/.mozilla/firefox/88nqvgl9.default')
firefox_profile.set_preference('browser.download.panel.shown', False)
firefox_profile.set_preference('browser.download.folderList', 2)
firefox_profile.set_preference('browser.download.dir', 
                               '/home/suzie/Documents/Data Warehousing/sdg_scraper/downloads')


options = Options()
options.add_argument('--headless')


driver=webdriver.Firefox(executable_path='/usr/bin/geckodriver', 
                         options=options , firefox_profile=firefox_profile)

actionChains = ActionChains(driver)

# getting all the goal urls
un_goals=['https://sustainabledevelopment.un.org/sdg{}'.format(i+1) for i in range(17)]

In [ ]:
# creating dictionaries
targets={'sdg_id':[], 'targ_id':[], 'desc':[]}
indicators={'sdg_id':[], 'targ_id':[], 'ind_id':[], 'desc':[]}
goals={'sdg_id':[],'desc':[]}


i=1
# loop through goals url:
for goal in un_goals:
    driver.get(goal)
    driver.find_element_by_css_selector("a[href='#targets']").click()
    
    element=driver.find_element_by_class_name("goal_text")
    soup=BeautifulSoup(element.get_attribute('innerHTML'))
    goals['desc'].append(soup.get_text().strip())
    goals['sdg_id'].append(str(i))
  
    element=driver.find_element_by_id('targets')
    soup=BeautifulSoup(element.get_attribute('innerHTML'))
    li = soup.find_all('div', attrs={'class': 'wrap'})[1:]
    
    for info in li:
        id_list = re.findall('<div class="mediumWhite">(.*?)</div>',str(info))
        
        indicator_ids = [ID.strip().split('.') for ID in id_list[1:]]
        indicator_sdg_ids = [ID[0] for ID in indicator_ids]
        indicator_targ_ids = [ID[1] for ID in indicator_ids]
        indicator_ind_ids = [ID[2] for ID in indicator_ids]
        indicators['sdg_id']+=indicator_sdg_ids
        indicators['targ_id']+=indicator_targ_ids
        indicators['ind_id']+=indicator_ind_ids
        
        target_ids = id_list[0].strip().split('.')
        keep_flag = False
        if len(target_ids) > 1:
            targets['sdg_id'].append(target_ids[0])
            targets['targ_id'].append(target_ids[1])
            keep_flag = True

        descs = re.findall('<div style="float:left;width:.*?">([\s\S]*?)</div>',str(info))
        descriptions = []
        for desc in descs:
            tag_list = re.findall('<div', desc)
            if len(tag_list) == 0:
                descriptions.append(re.sub('<.*?>','', desc))
                
        if keep_flag:
            targets['desc'].append(descriptions[0].strip())
        indicators['desc']+=filter(None, map(str.strip, descriptions[1:]))

    i+=1

In [ ]:
import pandas as pd
import csv

targets_df=pd.DataFrame(targets)
targets_df.to_csv('targets_CSV.csv', quoting=csv.QUOTE_ALL, index=False)
targets_df.style.set_properties(**{'text-align': 'left'})

,sdg_id,targ_id,desc
0,1,1,"By 2030, eradicate extreme poverty for all people everywhere, currently measured as people living on less than $1.25 a day"
1,1,2,"By 2030, reduce at least by half the proportion of men, women and children of all ages living in poverty in all its dimensions according to national definitions"
2,1,3,"Implement nationally appropriate social protection systems and measures for all, including floors, and by 2030 achieve substantial coverage of the poor and the vulnerable"
3,1,4,"By 2030, ensure that all men and women, in particular the poor and the vulnerable, have equal rights to economic resources, as well as access to basic services, ownership and control over land and other forms of property, inheritance, natural resources, appropriate new technology and financial services, including microfinance"
4,1,5,"By 2030, build the resilience of the poor and those in vulnerable situations and reduce their exposure and vulnerability to climate-related extreme events and other economic, social and environmental shocks and disasters"
5,1,a,"Ensure significant mobilization of resources from a variety of sources, including through enhanced development cooperation, in order to provide adequate and predictable means for developing countries, in particular least developed countries, to implement programmes and policies to end poverty in all its dimensions"
6,1,b,"Create sound policy frameworks at the national, regional and international levels, based on pro-poor and gender-sensitive development strategies, to support accelerated investment in poverty eradication actions"
7,2,1,"By 2030, end hunger and ensure access by all people, in particular the poor and people in vulnerable situations, including infants, to safe, nutritious and sufficient food all year round"
8,2,2,"By 2030, end all forms of malnutrition, including achieving, by 2025, the internationally agreed targets on stunting and wasting in children under 5 years of age, and address the nutritional needs of adolescent girls, pregnant and lactating women and older persons"
9,2,3,"By 2030, double the agricultural productivity and incomes of small-scale food producers, in particular women, indigenous peoples, family farmers, pastoralists and fishers, including through secure and equal access to land, other productive resources and inputs, knowledge, financial services, markets and opportunities for value addition and non-farm employment"


In [ ]:
indicators_df = pd.DataFrame(indicators)
indicators_df.to_csv('indicators_CSV.csv', quoting=csv.QUOTE_ALL, index=False)
indicators_df.style.set_properties(**{'text-align': 'left'})

,sdg_id,targ_id,ind_id,desc
0,1,1,1,"Proportion of population below the international poverty line, by sex, age, employment status and geographical location (urban/rural)"
1,1,2,1,"Proportion of population living below the national poverty line, by sex and age"
2,1,2,2,"Proportion of men, women and children of all ages living in poverty in all its dimensions according to national definitions"
3,1,3,1,"Proportion of population covered by social protection floors/systems, by sex, distinguishing children, unemployed persons, older persons, persons with disabilities, pregnant women, newborns, work-injury victims and the poor and the vulnerable"
4,1,4,1,Proportion of population living in households with access to basic services
5,1,4,2,"Proportion of total adult population with secure tenure rights to land, with legally recognized documentation and who perceive their rights to land as secure, by sex and by type of tenure"
6,1,5,1,"Number of deaths, missing persons and persons affected by disaster per 100,000 people"
7,1,5,2,Direct disaster economic loss in relation to global gross domestic product (GDP)a
8,1,5,3,Number of countries with national and local disaster risk reduction strategies
9,1,a,1,Proportion of resources allocated by the government directly to poverty reduction programmes


In [ ]:
goals_df=pd.DataFrame(goals)
goals_df.to_csv('goals_CSV.csv', quoting=csv.QUOTE_ALL, index=False)
goals_df.style.set_properties(**{'text-align': 'left'})

,sdg_id,desc
0,1,End poverty in all its forms everywhere
1,2,"End hunger, achieve food security and improved nutrition and promote sustainable agriculture"
2,3,Ensure healthy lives and promote well-being for all at all ages
3,4,Ensure inclusive and equitable quality education and promote lifelong learning opportunities for all
4,5,Achieve gender equality and empower all women and girls
5,6,Ensure availability and sustainable management of water and sanitation for all
6,7,"Ensure access to affordable, reliable, sustainable and modern energy for all"
7,8,"Promote sustained, inclusive and sustainable economic growth, full and productive employment and decent work for all"
8,9,"Build resilient infrastructure, promote inclusive and sustainable industrialization and foster innovation"
9,10,Reduce inequality within and among countries


In [ ]:
targets_df['desc'].str.len().max()

564

In [ ]:
indicators_df['desc'].str.len().max()

459

In [ ]:
goals_df['desc'].str.len().max()

188